# Diamond-Analysis-3


## Eval model w/ foreign inputs

The goal in this step is to integrate additional diamond data feeds and re-evaluate (1) our model and (2) the data source set for real world diamond estimation.   This new data Diamond_source2.csv comes from https://datasciencereview.com/case-study-does-the-size-of-diamonds-affect-its-price -- https://vincentarelbundock.github.io/Rdatasets/csv/Ecdat/Diamond.csv


### Eval 1. Carat linear regression


In [23]:
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from myutils import run_linear2
%matplotlib inline    

# train model
diamonds = pd.read_csv('diamonds-1a.csv')
diamonds = shuffle(diamonds, random_state=0)   # is this a repeatable shuffle by defautl w/o param
data = diamonds['carat'].to_frame()
target = diamonds['price'].to_frame()
regr = run_linear2(data, target, viz=False, log=False)

# eval w/ new data
diamonds2 = pd.read_csv('./Diamond_source2.csv')
print(diamonds2.head())

# col cleanup
colors = pd.get_dummies( diamonds2['colour'], prefix='color_')
clarity = pd.get_dummies( diamonds2['clarity'], prefix='clarity_')
price = diamonds2['price'].to_frame()
newdata = diamonds2['carat'].to_frame()
print(newdata.head())
print(newdata.columns)
print(data.columns)

diamonds_y_pred = regr.predict(newdata)

g = pd.DataFrame(diamonds_y_pred, columns=['predict'])
g['actual'] = price
g['carat'] = newdata
g['diff'] = g['actual'] - g['predict']
g['diff%'] = (abs(g['actual'] - g['predict'] ) / abs(g['actual'])) * 100

print('Score: ', regr.score(newdata, price))

g.sample(n=5, random_state=8)


Score:  0.9187743277836133
   Unnamed: 0  carat colour clarity certification  price
0           1   0.30      D     VS2           GIA   1302
1           2   0.30      E     VS1           GIA   1510
2           3   0.30      G    VVS1           GIA   1510
3           4   0.30      G     VS1           GIA   1260
4           5   0.31      D     VS1           GIA   1641
   carat
0   0.30
1   0.30
2   0.30
3   0.30
4   0.31
Index(['carat'], dtype='object')
Index(['carat'], dtype='object')
Score:  0.3030127734179965


,predict,actual,carat,diff,diff%
132,5577.733410,9853,1.01,4275.266590,43.390506
207,924.062851,1506,0.41,581.937149,38.641245
258,3406.020482,5333,0.73,1926.979518,36.133124
107,3561.142834,4667,0.75,1105.857166,23.695247
253,3173.336954,5510,0.70,2336.663046,42.407678


## Predictions and Score is shockingly low....  

But I think this new data is totally different since 1carat avg is totally different.  Did Kaggle give fake data, or is the other source fake data?   Below mean is skewed so low with Kaggle data that it underperforms on this new data


In [31]:
print('means: ', diamonds.loc[diamonds['carat'] == 1]['price'].mean(), ' vs ', diamonds2.loc[diamonds2['carat'] == 1]['price'].mean())
print('min: ', diamonds.loc[diamonds['carat'] == 1]['price'].min(), ' vs ', diamonds2.loc[diamonds2['carat'] == 1]['price'].min())
print('max: ', diamonds.loc[diamonds['carat'] == 1]['price'].max(), ' vs ', diamonds2.loc[diamonds2['carat'] == 1]['price'].max())
print('std: ', diamonds.loc[diamonds['carat'] == 1]['price'].std(), ' vs ', diamonds2.loc[diamonds2['carat'] == 1]['price'].std())

diamonds.loc[diamonds['carat'] == 1]['price'].describe()


means:  5243.005141388175  vs  10188.8125
min:  1681  vs  7818
max:  16469  vs  15582
std:  1604.0840477131762  vs  1908.05103126285


count     1556.000000
mean      5243.005141
std       1604.084048
min       1681.000000
25%       4155.000000
50%       4864.000000
75%       6079.000000
max      16469.000000
Name: price, dtype: float64

Hmm is it even worth continuing..

### Eval 2. Multiple linear regression

In [6]:
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from myutils import run_linear2
%matplotlib inline    

diamonds = pd.read_csv('diamonds-1a.csv')
diamonds = shuffle(diamonds, random_state=0)   # is this a repeatable shuffle by defautl w/o param
data = diamonds['carat'].to_frame()
target = diamonds['price'].to_frame()
regr = run_linear2(data, target, viz=False, log=False)

diamonds2 = pd.read_csv('./Diamond_source2.csv')
print(diamonds2.head())

# col cleanup
colors = pd.get_dummies( diamonds2['colour'], prefix='color_')
clarity = pd.get_dummies( diamonds2['clarity'], prefix='clarity_')
price = diamonds2['price'].to_frame()
diamonds2 = diamonds2.join(colors).join(clarity)
diamonds2 = diamonds2.drop(columns=['price','colour','clarity','certification','Unnamed: 0'])
print(diamonds2.head())

# zero filling the extra cols
diamonds2.insert(1,'cut__Very Good',0)
diamonds2.insert(1,'cut__Premium',0)
diamonds2.insert(1,'cut__Ideal',0)
diamonds2.insert(1,'cut__Good',0)
diamonds2.insert(1,'cut__Fair',0)
diamonds2.insert(12,'color__J',0)
diamonds2.insert(13,'clarity__I1',0)
diamonds2.insert(15,'clarity__SI2',0)
diamonds2.insert(15,'clarity__SI1',0)
print(diamonds2.head())
print(diamonds2.columns)
print(data.columns)

# diamonds_y_pred = regr.predict(diamonds2)
# res = pd.DataFrame(diamonds_y_pred)
# res['actual'] = price

# print('Score: ', regr.score(diamonds2, price))

#Y = regr.predict(X)


Score:  0.9187743277836133
   Unnamed: 0  carat colour clarity certification  price
0           1   0.30      D     VS2           GIA   1302
1           2   0.30      E     VS1           GIA   1510
2           3   0.30      G    VVS1           GIA   1510
3           4   0.30      G     VS1           GIA   1260
4           5   0.31      D     VS1           GIA   1641
   carat  color__D  color__E  color__F  color__G  color__H  color__I  \
0   0.30         1         0         0         0         0         0   
1   0.30         0         1         0         0         0         0   
2   0.30         0         0         0         1         0         0   
3   0.30         0         0         0         1         0         0   
4   0.31         1         0         0         0         0         0   

   clarity__IF  clarity__VS1  clarity__VS2  clarity__VVS1  clarity__VVS2  
0            0             0             1              0              0  
1            0             1             0      